In [138]:
# Fix: Add list handling to expand_spec function
from itertools import product, combinations
from collections.abc import Mapping

def expand_spec_fixed(node):
    # NEW: Handle lists by expanding each element and taking the product
    if isinstance(node, list):
        if not node:
            return [[]]  # Empty list -> single empty result

        # Special case: if there's only one element in the list, check if we should unwrap
        if len(node) == 1:
            element_result = expand_spec_fixed(node[0])

            # If the result contains lists (combinations), return directly to prevent extra wrapping
            # If the result contains scalars, we still need to wrap them properly
            if element_result and isinstance(element_result[0], list):
                return element_result
            # Otherwise, fall through to normal processing

        # Expand each element in the list
        expanded_elements = [expand_spec_fixed(element) for element in node]

        # Take Cartesian product of all expansions
        results = []
        for combo in product(*expanded_elements):
            results.append(list(combo))  # Convert tuple to list
        return results

    # Rest of the logic remains the same for dictionaries
    if not isinstance(node, Mapping):
        return [node]

    # Case 1: pure OR node (with optional size)
    if set(node.keys()) == {"or"} or set(node.keys()) == {"or", "size"}:
        choices = node["or"]
        size = node.get("size", None)

        if size is not None:
            # Apply size constraints first, then expand
            out = []

            # Handle tuple size (from, to) or single size
            if isinstance(size, tuple) and len(size) == 2:
                from_size, to_size = size
                # Generate combinations for all sizes from from_size to to_size (inclusive)
                for s in range(from_size, to_size + 1):
                    if s > len(choices):
                        continue
                    for combo in combinations(choices, s):
                        # Expand this specific combination and return as individual elements
                        combo_results = _expand_combination(combo)
                        out.extend(combo_results)
            else:
                # Single size value
                if size <= len(choices):
                    for combo in combinations(choices, size):
                        # Expand this specific combination and return as individual elements
                        combo_results = _expand_combination(combo)
                        out.extend(combo_results)

            return out
        else:
            # Original behavior: expand all choices
            out = []
            for choice in choices:
                out.extend(expand_spec_fixed(choice))
            return out

    # Case 2: dict that also contains "or" -> branch and merge
    if "or" in node:
        # Extract size if present
        size = node.get("size", None)
        base = {k: v for k, v in node.items() if k not in ["or", "size"]}
        base_expanded = expand_spec_fixed(base)            # list[dict]

        # Create a temporary or node with size
        or_node = {"or": node["or"]}
        if size is not None:
            or_node["size"] = size

        choice_expanded = expand_spec_fixed(or_node)  # list[dict or scalar]
        results = []
        for b in base_expanded:
            for c in choice_expanded:
                if isinstance(c, Mapping):
                    merged = {**b, **c}
                    results.append(merged)
                else:
                    # Scalar choices only make sense as values under a key, not top-level merges
                    raise ValueError("Top-level 'or' choices must be dicts.")
        return results

    # Case 3: normal dict -> product over keys
    keys, options = zip(*[(k, _expand_value_fixed(v)) for k, v in node.items()]) if node else ([], [])
    if not keys:
        return [{}]
    out = []
    for combo in product(*options):
        d = {}
        for k, v in zip(keys, combo):
            d[k] = v
        out.append(d)
    return out

def _expand_combination(combo):
    """Expand a specific combination of choices by taking their cartesian product."""
    expanded_choices = []
    for choice in combo:
        expanded_choices.append(expand_spec_fixed(choice))

    # Take cartesian product
    results = []
    for expanded_combo in product(*expanded_choices):
        # Return the combination as a single list (flatten one level)
        results.append(list(expanded_combo))

    return results

def _expand_value_fixed(v):
    # Value position returns a list of *values* (scalars or dicts)
    if isinstance(v, Mapping) and ("or" in v.keys()):
        # Value-level OR can yield scalars or dicts as values (with optional size)
        choices = v["or"]
        size = v.get("size", None)

        if size is not None:
            # Apply size constraints first, then expand
            vals = []

            # Handle tuple size (from, to) or single size
            if isinstance(size, tuple) and len(size) == 2:
                from_size, to_size = size
                # Generate combinations for all sizes from from_size to to_size (inclusive)
                for s in range(from_size, to_size + 1):
                    if s > len(choices):
                        continue
                    for combo in combinations(choices, s):
                        # Expand this specific combination
                        combo_results = _expand_combination(combo)
                        vals.extend(combo_results)
            else:
                # Single size value
                if size <= len(choices):
                    for combo in combinations(choices, size):
                        # Expand this specific combination
                        combo_results = _expand_combination(combo)
                        vals.extend(combo_results)

            return vals
        else:
            # Original behavior: expand all choices
            vals = []
            for choice in choices:
                ex = expand_spec_fixed(choice)
                # expand_spec returns list; extend with each item (scalar or dict value)
                vals.extend(ex)
            return vals
    elif isinstance(v, Mapping):
        # Nested object: expand to list of dict values
        return expand_spec_fixed(v)
    elif isinstance(v, list):
        # Handle lists in value positions
        return expand_spec_fixed(v)
    else:
        return [v]

In [139]:
pipeline_config = [
        {"or": [None, "A", "B"]},  # scale the features
        [
            {"or": ["a", "b"]},
            None,
            [{"or": ["1", "2"]}, {"or": ["x", "y"]}],
        ]
    ]

results = expand_spec_fixed(pipeline_config)
print(f"Number of combinations (24): {len(results)}")
# for i, cfg in enumerate(results):
    # print(f"  {i+1}: {cfg}")

Number of combinations (24): 24


In [140]:
pipeline_config_with_size = [
        {"or": [None, "A", "B", "C", "D"], "size": 4},  # scale the features
        [
            {"or": ["a", "b"]},
            None,
            [{"or": ["1", "2"]}, {"or": ["x", "y"]}],
        ]
    ]

results_with_size = expand_spec_fixed(pipeline_config_with_size)
# print(f"Expected: C(5,3) = 10 c(5,4) = 5")
print(f"Number of combinations (40): {len(results_with_size)}")
# for i, cfg in enumerate(results_with_size):
    # print(f"  {i+1}: {cfg}")

Number of combinations (40): 40


In [141]:
pipeline_config_with_tuple_size = [
        {"or": [None, "A", "B", "C", "D"], "size": (4, 5)},  # scale the features
        [
            {"or": ["a", "b"]},
            None,
            [{"or": ["1", "2"]}, {"or": ["x", "y"]}],
        ]
    ]

results_with_tuple_size = expand_spec_fixed(pipeline_config_with_tuple_size)
# print(f"Expected: C(5,3) + C(5,4) + C(5,5) = 10 + 5 + 1 = 16 combinations (before multiplying by other expansions)")
print(f"Number of combinations (48): {len(results_with_tuple_size)}")
# for i, cfg in enumerate(results_with_tuple_size):
    # print(f"  {i+1}: {cfg}")

Number of combinations (48): 48


In [142]:
pipeline_config_with_tuple_size = [
        {"or": [None, "A", "B", "C", {"or": ["a", "b"]}], "size": 4},  # scale the features
    ]
results_with_tuple_size = expand_spec_fixed(pipeline_config_with_tuple_size)
print(f"Number of combinations: {len(results_with_tuple_size)}")
# for i, cfg in enumerate(results_with_tuple_size):
    # print(f"  {i+1}: {cfg}")

pipeline_config_with_tuple_size = [
        {"or": [None, "A", "B", "C", {"or": ["a", "b"]}], "size": (3, 4)},  # scale the features
    ]
results_with_tuple_size = expand_spec_fixed(pipeline_config_with_tuple_size)
print(f"Number of combinations: {len(results_with_tuple_size)}")
# for i, cfg in enumerate(results_with_tuple_size):
    # print(f"  {i+1}: {cfg}")

Number of combinations: 9
Number of combinations: 25


In [150]:
p = [{"or": ["A", "B", "C"], "size": (1, 3)}]
r = expand_spec_fixed(p)
for item in r:
    print(item)

p = [{"or": ["A", "B", "C"]}]
r = expand_spec_fixed(p)
for item in r:
    print(item)

['A']
['B']
['C']
['A', 'B']
['A', 'C']
['B', 'C']
['A', 'B', 'C']
['A']
['B']
['C']


In [148]:
pipeline = [
    # {"or":['MinMaxScaler', 'StandardScaler']},
    {"feature": {"or":["None", "savgol", "snv", "msc", "haar", "gaussian"], "size": (1,6)}}
]

r = expand_spec_fixed(pipeline)
print(f"Number of combinations: {len(r)}")
for i, cfg in enumerate(r):
    print(f"  {i+1}: {cfg}")

Number of combinations: 63
  1: [{'feature': ['None']}]
  2: [{'feature': ['savgol']}]
  3: [{'feature': ['snv']}]
  4: [{'feature': ['msc']}]
  5: [{'feature': ['haar']}]
  6: [{'feature': ['gaussian']}]
  7: [{'feature': ['None', 'savgol']}]
  8: [{'feature': ['None', 'snv']}]
  9: [{'feature': ['None', 'msc']}]
  10: [{'feature': ['None', 'haar']}]
  11: [{'feature': ['None', 'gaussian']}]
  12: [{'feature': ['savgol', 'snv']}]
  13: [{'feature': ['savgol', 'msc']}]
  14: [{'feature': ['savgol', 'haar']}]
  15: [{'feature': ['savgol', 'gaussian']}]
  16: [{'feature': ['snv', 'msc']}]
  17: [{'feature': ['snv', 'haar']}]
  18: [{'feature': ['snv', 'gaussian']}]
  19: [{'feature': ['msc', 'haar']}]
  20: [{'feature': ['msc', 'gaussian']}]
  21: [{'feature': ['haar', 'gaussian']}]
  22: [{'feature': ['None', 'savgol', 'snv']}]
  23: [{'feature': ['None', 'savgol', 'msc']}]
  24: [{'feature': ['None', 'savgol', 'haar']}]
  25: [{'feature': ['None', 'savgol', 'gaussian']}]
  26: [{'featu